In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
base_model = tf.keras.models.load_model('/content/drive/MyDrive/my_network_model.h5')
base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 256)               20480     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_17 (Dense)            (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 64)                8256      
                                                                 
 dense_19 (Dense)            (None, 6)                 390       
                                                                 
Total params: 62022 (242.27 KB)
Trainable params: 62022 

In [4]:
input_shape = base_model.layers[0].input_shape
print(input_shape)

(None, 79)


In [5]:
path = '/content/drive/My Drive/'
df = pd.read_csv(path + "cicids2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,22,1266342,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,22,1319353,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,22,160,1,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,22,1303488,41,42,2728,6634,456,0,66.536585,110.129945,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,35396,77,1,2,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [6]:
df.drop(["Flow Bytes/s"," Flow Packets/s"],axis=1,inplace=True)

In [7]:
features = df.columns.tolist()
features.remove(" Label")
X = df[features]
y=df[' Label']

In [8]:
X

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,22,1266342,41,44,2664,6954,456,0,64.975610,109.864573,...,24,32,0.0,0.0,0,0,0.0,0.0,0,0
1,22,1319353,41,44,2664,6954,456,0,64.975610,109.864573,...,24,32,0.0,0.0,0,0,0.0,0.0,0,0
2,22,160,1,1,0,0,0,0,0.000000,0.000000,...,0,32,0.0,0.0,0,0,0.0,0.0,0,0
3,22,1303488,41,42,2728,6634,456,0,66.536585,110.129945,...,24,32,0.0,0.0,0,0,0.0,0.0,0,0
4,35396,77,1,2,0,0,0,0,0.000000,0.000000,...,0,32,0.0,0.0,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286462,443,196135,49,57,1331,105841,570,0,27.163265,108.067176,...,31,20,0.0,0.0,0,0,0.0,0.0,0,0
286463,443,378424,49,59,1325,104393,570,0,27.040816,108.095051,...,30,20,0.0,0.0,0,0,0.0,0.0,0,0
286464,443,161800,70,103,1427,215903,570,0,20.385714,90.746389,...,47,20,0.0,0.0,0,0,0.0,0.0,0,0
286465,443,142864,50,62,1331,110185,570,0,26.620000,107.027727,...,31,20,0.0,0.0,0,0,0.0,0.0,0,0


In [12]:
# remove the original input layer b/c mismatch
base_output = base_model.layers[1].output
for layer in base_model.layers[2:]:
    base_output = layer(base_output)

# new input layer matching the cicids2017 data
new_input = Input(shape=(76,))

# rbuild model from the first dense layer, excluding the original input layer
x = tf.keras.layers.Dense(256, activation='relu')(new_input)  # next layer apparently  had 256 units

# attach remaining layers
for layer in base_model.layers[2:]:
    x = layer(x)

new_model = Model(inputs=new_input, outputs=x)
new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 76)]              0         
                                                                 
 dense_1 (Dense)             (None, 256)               19712     
                                                                 
 dense_17 (Dense)            (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 64)                8256      
                                                                 
 dense_19 (Dense)            (None, 6)                 390       
                                                                 
Total params: 61254 (239.27 KB)
Trainable params: 61254 (23

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

# scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# encode labels
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

history = new_model.fit(X_train_scaled, y_train_encoded, epochs=20, batch_size=32, validation_split=0.2)

test_loss, test_acc = new_model.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Accuracy: {test_acc}")

Epoch 1/20
5372/5372 [==============================] - 34s 6ms/step - loss: 0.0254 - accuracy: 0.9957 - val_loss: 0.0047 - val_accuracy: 0.9989
Epoch 2/20
5372/5372 [==============================] - 32s 6ms/step - loss: 0.0089 - accuracy: 0.9984 - val_loss: 0.0047 - val_accuracy: 0.9984
Epoch 3/20
5372/5372 [==============================] - 27s 5ms/step - loss: 0.0067 - accuracy: 0.9990 - val_loss: 0.0029 - val_accuracy: 0.9989
Epoch 4/20
5372/5372 [==============================] - 28s 5ms/step - loss: 0.0050 - accuracy: 0.9992 - val_loss: 0.0020 - val_accuracy: 0.9995
Epoch 5/20
5372/5372 [==============================] - 25s 5ms/step - loss: 0.0061 - accuracy: 0.9992 - val_loss: 0.0020 - val_accuracy: 0.9998
Epoch 6/20
5372/5372 [==============================] - 23s 4ms/step - loss: 0.0046 - accuracy: 0.9995 - val_loss: 0.0019 - val_accuracy: 0.9998
Epoch 7/20
5372/5372 [==============================] - 27s 5ms/step - loss: 0.0038 - accuracy: 0.9995 - val_loss: 0.0067 - val_ac